In [2]:
import sys
sys.path.append("../src/")

from glob import glob
from os.path import join, splitext, basename

import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

from global_config import FOLDER_IMAGES, FOLDER_MRKS, IMAGE_SIZE, RANDOM_SEED
from mrk_file import MRKFile
from utils import load, plot

%matplotlib inline

In [3]:
STAMP_SIZE = (50, 50)
BATCH_SIZE = 32

PATH_TO_ROOT = "../"
FOLDER_DATA = join(PATH_TO_ROOT, "data")
FOLDER_MRKS = join(PATH_TO_ROOT, FOLDER_MRKS)
FOLDER_IMAGES = join(PATH_TO_ROOT, FOLDER_IMAGES)

print(FOLDER_DATA)
print(FOLDER_IMAGES)
print(FOLDER_MRKS)

../data
../data/cropped_faces/images/
../data/cropped_faces/ground_truth/


# Data Loading 

In [4]:
mrk_path = lambda filepath: join(FOLDER_MRKS, splitext(basename(filepath))[0]) + ".mrk"

image_files = glob(FOLDER_IMAGES + '*')
mrk_files = np.array([mrk_path(path) for path in image_files])

print(len(image_files))
print(len(mrk_files))
assert(len(image_files) == len(mrk_files))

5731
5731


In [5]:
images = load.images_from_list_files(image_files, output_size=IMAGE_SIZE)
mrks = load.mrk_files_from_list_files(mrk_files)
print(mrks.shape, mrks.dtype)

5731 of 5731
5731 of 5731
(5731,) object


# Image Data Generation

In [22]:
datagen = ImageDataGenerator(
    rotation_range=90,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

In [40]:
flow = datagen.flow_from_directory(
    FOLDER_DATA,
    target_size=STAMP_SIZE,
    color_mode="rgba",
    classes=["stamps"],
    batch_size=BATCH_SIZE,
    seed=RANDOM_SEED,
    interpolation="bicubic",
)

Found 32 images belonging to 1 classes.


In [47]:
def rect_contains(rect, point):
    top, left, bottom, right = rect
    x, y = point
    return top < y < (top + bottom) and left < x < (left + right)


def random_point_out_of_rect(rect, max_x, max_y):
    random_x, random_y = rect[0] + 1, rect[1] + 1
    while rect_contains(rect, (random_x, random_y)):
        random_x = np.random.choice(max_x)
        random_y = np.random.choice(max_y)
    return random_x, random_y


def create_foreground_image(stamp, height, width, inner_rect):
    im_fgd = np.zeros((height, width, 4))
    stamp_height, stamp_width = stamp.shape[0], stamp.shape[1]
    
    random_x, random_y = random_point_out_of_rect(
        inner_rect,
        width - stamp_width - 1, 
        height - stamp_height - 1,
    )
    
    top, left = random_y, random_x
    bottom, right = top + stamp_height, left + stamp_width
    im_fgd[top:bottom, left:right, :] = stamp
    return im_fgd


def alpha_blending(im_foreground, im_background):
    c1, c2, c3, ca = cv2.split(im_foreground)
    im_fgd = cv2.merge((c1, c2, c3)).astype(np.float)
    im_bgd = im_background.astype(np.float)
    alpha = cv2.merge((ca, ca, ca)).astype(np.float) / 255

    im_fgd = cv2.multiply(alpha, im_fgd)
    im_bgd = cv2.multiply(1.0 - alpha, im_bgd)
    im_out = cv2.add(im_fgd, im_bgd)
    return im_out


def add_stamp_to_image(image, stamp, inner_rect, border_size=25):
    im_bgd = cv2.copyMakeBorder(
        image,
        top=border_size,
        bottom=border_size,
        left=border_size,
        right=border_size,
        borderType=cv2.BORDER_CONSTANT,
        value=0,
    )
    
    im_fgd = create_foreground_image(stamp, im_bgd.shape[0], im_bgd.shape[1], inner_rect)
    im_out = alpha_blending(im_fgd, im_bgd)
    
    top = left = border_size
    bottom = top + image.shape[0]
    right = left + image.shape[1]
    
    return im_out[top:bottom, left:right]

In [52]:
im = images[np.random.choice(images.shape[0])]
stamp = flow.next()[0][np.random.choice(32)]
im = add_stamp_to_image(im, stamp, (40, 40, 120, 120))
cv2.imwrite("stamp.png", stamp)
cv2.imwrite("out.png", im)

True

In [ ]:
N_IMAGES = images.shape[0]
batch_idxs = np.split(range(N_IMAGES), range(BATCH_SIZE, N_IMAGES, BATCH_SIZE))

for idxs in batch_idxs:
    half_size = len(idxs) // 2
    random_idxs = np.random.choice(idxs, size=half_size, replace=False)
    random_stamps = np.random.choice(BATCH_SIZE, size=half_size, replace=False)
    
    current_images = images[random_idxs]
    current_mrks = mrks[random_idxs]
    gen_stamps = flow.next()[0][random_stamps]
    print(random_idxs)
    print(current_images.shape)
    print(gen_stamps.shape)
    break